# Introduction
This dataset comes from the NASA program SLAP(Scanning L-Band Active Passive). Their main goal is to measure soil moisture using 
temperature readings. These readings are taken using a device that is attached to a plane which flies in a rectangular pattern to capture a pre-determined plot of earth. The device used to take readings utilizes an oscialliting sensor that takes readings in an eliptical pattern. 

There is a need for this group to accurately determine which areas are land and which are water. This is mainly found through temperature readings, and the algorithms they use are inaccurate due to the fact they are mainly using google earth and guessing by eye (Husband has verified this to be true). 

For this project, I will be using KMeans to assign new flags for water. My goal is to identify at least 3 values (water, shoreline, and dry land).



In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import folium
from matplotlib import cm
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Data
Dataset link: https://earth.gsfc.nasa.gov/hydro/instruments/slap/campaigns/slapex-liaise-campaign-spain-2021

The dataset I am using is from a campaign in 2021 called "Land surface Interactions with the Atmosphere over the Iberian Semi-arid Environment" (LIAISE). This LIASE campaign had a total of 9 flights with a mix of low and medium altitudes. 

I chose the 17 July 2021, low altitude cleaned version (if you look in the L2_SM_P column). The SLAP group cleans the data for radio interference and adds flags for water, among other things. 

This dataset has a shape of 320855, 38.

In [2]:
path = "C:/Users/ashkl/OneDrive/Documents/Loyola/machine learning/SLAP.csv"

In [3]:
df = pd.read_csv(path)
df.shape

(320855, 38)

# Data Cleaning

I will be removing an NA values. The first portion of the data is actually a set of calibrations that are done on land. If you see the below, there are 137,852 NA values for footprint_lat and footprint_lon, so I will remove those, then there was still a few hiding in the NEDT_v column, so I removed them as well. I will also need to ensure there are no blank, hidden spaces in the column names. After NA removal, I am left with 182,419 rows which I believe will be plenty to achieve my goal.

In [4]:
df.isna().sum()

 TBh                             74976
 TBv                             74958
 TB_IR                               0
 NEDT_h                          74976
 NEDT_v                          74958
 footprint_lat                  137852
 footprint_lon                  137852
 footprint_semimajor             88431
 footprint_semiminor             88431
 footprint_semimajor_azimuth    137852
 aircraft_lat                        0
 aircraft_lon                        0
 aircraft_alt_MSL                    0
 aircraft_alt_AGL                    0
 aircraft_roll_angle             46163
 aircraft_pitch_angle            46163
 aircraft_heading                46803
 flag_RFI_time_domain_h              0
 flag_RFI_time_domain_v              0
 flag_RFI_cross_frequency_h          0
 flag_RFI_cross_frequency_v          0
 flag_RFI_kurtosis_h                 0
 flag_RFI_kurtosis_v                 0
 flag_contam_h                       0
 flag_contam_v                       0
 flag_flight             

In [5]:
df.columns = df.columns.str.replace(' ', '')
df = df[df['footprint_lat'].notna() & df['NEDT_v'].notna()]
print(df.isna().sum(), df.shape)
#TBh and TBv are horizontal and vertical polarization. They are from 2 diff mechanisms and take slightly diff readings. 

TBh                            0
TBv                            0
TB_IR                          0
NEDT_h                         0
NEDT_v                         0
footprint_lat                  0
footprint_lon                  0
footprint_semimajor            0
footprint_semiminor            0
footprint_semimajor_azimuth    0
aircraft_lat                   0
aircraft_lon                   0
aircraft_alt_MSL               0
aircraft_alt_AGL               0
aircraft_roll_angle            0
aircraft_pitch_angle           0
aircraft_heading               0
flag_RFI_time_domain_h         0
flag_RFI_time_domain_v         0
flag_RFI_cross_frequency_h     0
flag_RFI_cross_frequency_v     0
flag_RFI_kurtosis_h            0
flag_RFI_kurtosis_v            0
flag_contam_h                  0
flag_contam_v                  0
flag_flight                    0
flag_box_calibration           0
flag_water_calibration         0
flag_temperature_stable_h      0
flag_temperature_stable_v      0
flag_anten

In [6]:
df = df[['TBh', 'TBv', 'TB_IR', 'footprint_lat', 'footprint_lon', 'footprint_semimajor', 'footprint_semiminor', 
           'footprint_semimajor_azimuth', 'aircraft_alt_MSL', 'aircraft_alt_AGL', 'flag_RFI_time_domain_h', 'flag_RFI_time_domain_v',
          'flag_temperature_stable_h', 'flag_temperature_stable_v', 'flag_water', 'flag_contam_h', 'flag_contam_v']]

In [8]:
df.head()


,TBh,TBv,TB_IR,footprint_lat,footprint_lon,footprint_semimajor,footprint_semiminor,footprint_semimajor_azimuth,aircraft_alt_MSL,aircraft_alt_AGL,flag_RFI_time_domain_h,flag_RFI_time_domain_v,flag_temperature_stable_h,flag_temperature_stable_v,flag_water,flag_contam_h,flag_contam_v
59309,262.38,284.83,294.27,39.531,-0.41914,70.323,33.307,30.001,308.39,253.39,1,1,1,1,0,1,1
59310,263.18,285.42,294.93,39.531,-0.41914,70.333,33.312,28.944,308.43,253.43,0,1,1,1,0,1,1
59311,271.41,287.66,294.78,39.531,-0.41914,70.344,33.318,27.757,308.47,253.47,0,1,1,1,0,1,1
59312,266.14,295.15,294.93,39.531,-0.41914,70.355,33.323,26.469,308.51,253.51,0,1,1,1,0,1,1
59313,271.18,292.41,294.95,39.531,-0.41916,70.366,33.328,25.048,308.55,253.55,0,0,1,1,0,1,1


# Feature Engineering and Data Exploration

In this section, I am exploring the data and looking at correlation, checking which features might be best to keep, and checking the temperature distributions. 

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='rocket_r')

## Checking for best features using SelectKBest

In [ ]:
X = df.drop(['flag_water'], axis=1)
y = df.flag_water

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

# Borrowing some code here from homework 8
bestfeatures = SelectKBest(score_func=f_regression, k=4)
fit = bestfeatures.fit(X_train,y_train)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df.columns)

# Since I want to see all of the scores, I changed the printout to include all. 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feature','Score']
print(featureScores.nlargest(17,'Score'))

In [ ]:
#keep anything with k-score greater than or equal to 2000, then check out the new df
df_high_Kscore = df.drop(df.columns[[12, 13, 2, 10, 14, 15, 7, 11, 16]], axis=1)
df_high_Kscore.head()

## Looking at the statistics of the kept columns

We can see some pretty intersting stats. There is definitely some skew in a few of the features like the TBh and TBv values I want to use to see if we can make new groups of water flags. In the boxplot, we can see this skewness a bit easier.

In [ ]:
#let's see some stats.
df_high_Kscore.describe()

In [ ]:
xticks = list(df_high_Kscore.columns)
p = plt.boxplot(df_high_Kscore, labels = xticks, vert = False)
plt.title('Box and Whisker Plots of High-socring Features', fontsize = 14)
plt.xlabel("Reading Value", fontsize = 12)
plt.ylabel("Feature", fontsize = 12)
;

## Taking a closer look at the TBh and TBv temperature values

Now, we can see clearly that our temperature readings (TBh and TBv) both have some issues with outliers. I have been told this is due to the readings mainly taking place on land, so there is a naturally higher temp with land readings. See the distribution graphs below for a better look. The footprint latitude and longitude, we can ignore for now. The two semiminor and semimajor footprints as well as the aircraft_alt_AGL look good, but the aircraft_alt_MSL has some skew. Let's take a closer look at the temp readings.

In [ ]:
sns.histplot(data=df_high_Kscore, x="TBh", color='lightblue', stat='density')
sns.kdeplot(data=df_high_Kscore, x="TBh", color='black')
plt.title('Distribution Plot of TBh Reading', fontsize = 14)
plt.xlabel("TBh Value", fontsize = 12)
plt.ylabel("Count", fontsize = 12);
#need to plot just water or just land

In [ ]:
sns.histplot(data=df_high_Kscore, x="TBv", color='lightblue', stat='density')
sns.kdeplot(data=df_high_Kscore, x="TBv", color='black')
plt.title('Distribution Plot of TBv Reading', fontsize = 14)
plt.xlabel("TBv Value", fontsize = 12)
plt.ylabel("Count", fontsize = 12);

Based on the distribution plots of the temperature readings, we can see that there are at least 3, possibly 4 modals if we look at the TBh reading. 

## Map of readings

By looking at the map, we can see a picture of where the water/land is and get a good perspective of the problem.

In [ ]:
#get center of map so we can see the map
import statistics
print("latitude: ", statistics.mean(df["footprint_lat"]))
print("longitude: ", statistics.mean(df["footprint_lon"]))
map_df = df[['footprint_lat', 'footprint_lon']].copy()
center_of_map = [41.039595601335385, 0.6624781636930398] 

In [ ]:

my_map = folium.Map(location = center_of_map,
                   zoom_start = 7,
                   width = '90%',
                   height = '100%',
                   left = '5%',
                   right = '5%',
                   top = '0%',
                    tiles="CartoDB Positron") 
for index, row in map_df.iterrows():
    folium.Circle(location=[row['footprint_lat'], row['footprint_lon']],
                  radius=50,
                  color='black',
                  fill=True,
                  fill_color="black",
                  fill_opacity=0.5).add_to(my_map)
# my_map.save(path + 'Dots_map_only.html')
my_map


# Analysis

Trying different parameters and sub-sets of data. I have a hunch that only using TBh and TBv will give me the best results, but I will try other possiblities as well. 

## TBh and TBv only
Starting with these two variables since I believe this will give the best results. These are Temperatier Brightness readings collected from a Horizontal-facing sensor and a Vertical-facing sensor.

In [ ]:
#scale temperatures
sc = StandardScaler()
df_high_Kscore[['T_TBh', 'T_TBv']] = sc.fit_transform(df_high_Kscore[['TBh', 'TBv']])

#create array of values
X = df_high_Kscore[['T_TBh', 'T_TBv']].values


In [ ]:
#do an elbow plot and determine number of clusters
def optimise_k_means(data, max_k):
    means=[]
    inertias=[]
    
    for k in range(1, max_k):
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(data)        
        means.append(k)
        inertias.append(kmeans.inertia_)
    fig=plt.subplots(figsize = (10, 5))
    plt.plot(means, inertias, 'o-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()

optimise_k_means(df_high_Kscore[['T_TBh', 'T_TBv']], 10)

In [ ]:
#preliminary run of model
km = KMeans(n_clusters=3, 
            init='random', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)


In [ ]:
#setup a df for a map with cluster information
map_df = df[['footprint_lat', 'footprint_lon']].copy()
map_df['cluster'] = km.labels_

map_df.head()

In [ ]:
#map of preliminary clusters
my_map = folium.Map(location = center_of_map,
                   zoom_start = 7,
                   width = '90%',
                   height = '100%',
                   left = '5%',
                   right = '5%',
                   top = '0%',
                    tiles="CartoDB Positron") 
for index, row in map_df.iterrows():
    if row['cluster'] == 0:
        color = 'green'
    elif row['cluster'] == 1:
        color = 'brown'       
    else:
        color = 'blue' 
    folium.Circle(location=[row['footprint_lat'], row['footprint_lon']],
                  radius=50,
                  color=color,
                  fill=True,
                  fill_color=color,
                  fill_opacity=0.5).add_to(my_map)
# my_map.save(path + 'Dots_map_preliminary.html')
my_map

In [ ]:
#preliminary silhoutte graphic

# cluster_labels = np.unique(y_km)
# n_clusters = cluster_labels.shape[0]
# silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
# y_ax_lower, y_ax_upper = 0, 0
# yticks = []
# for i, c in enumerate(cluster_labels):
#     c_silhouette_vals = silhouette_vals[y_km == c]
#     c_silhouette_vals.sort()
#     y_ax_upper += len(c_silhouette_vals)
#     color = cm.jet(float(i) / n_clusters)
#     plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
#              edgecolor='none', color=color)

#     yticks.append((y_ax_lower + y_ax_upper) / 2.)
#     y_ax_lower += len(c_silhouette_vals)
    
# silhouette_avg = np.mean(silhouette_vals)
# plt.axvline(silhouette_avg, color="red", linestyle="--") 

# plt.yticks(yticks, cluster_labels + 1)
# plt.ylabel('Cluster')
# plt.xlabel('Silhouette coefficient')

# plt.tight_layout()
# plt.savefig('silhoutte plot 1.png', dpi=300)
# plt.show()

IN the above silhoutte graphic, we can see that each cluster has vastly different height, but their lengths are somewhat close together.

In [ ]:
#ran a gridsearch for kicks. I know it doesn't really work for unsupervised learning, but it does give me some ideas!

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

param_grid = {'n_clusters': [3],  
              'init':["random", "k-means++"],
              'n_init':["auto", 3, 10, 20],
              'max_iter':[100, 300, 500],
              'algorithm':["lloyd", "elkan"],
             }  
   
grid = GridSearchCV(KMeans(), param_grid, refit = True, verbose = False,n_jobs=3) 
   
grid.fit(X) 
 
print(grid.best_params_) 


In the following cell, I run the KMeans with different possible combinations of hyper-parameters. For each, I print out the number of datapoints in each cluster and the silhoutte score. 

In [ ]:
#Score for preliminary test run
km = KMeans(n_clusters=3, 
            init='random', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)
# print("Preliminary model score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#change init=k-means++
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)
# print("Init= k-means++ change score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())


#change max iterations to 100
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=100,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)
# print("max iteration '100' change score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#change max iteration to 500
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)
# print("max iteration '500' change score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#change algorithm
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0,
            algorithm='elkan')

y_km = km.fit_predict(X)
# print("algorithm 'elkan' change score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())


We can see that for just TBh and TBv, the best model so far is actually a tie between using 100 and 500 iterations with k-means++ and the default algorithm.
- Silhoutte score:  0.828039746880101
- counts of each cluster:
    - 1  ||  142603
    - 0  ||   30649
    - 2  ||    9167

## TB_IR included with TBh and TBv

Next, I include the TB_IR which is the brightness temperature measured by the TIR radiometer in kelvins. This temperature reading is most similar to your every-day infrared thermometer you might use to take your body temperature.

In [ ]:
#create dataframe for all the temperature reading types.
temperatures_df = df[['TBh', 'TBv', 'TB_IR']]

#transform the values. This is especially needed here due to the different types of measurements being used.
temperatures_df[['T_TBh', 'T_TBv', 'T_TB_IR']] = sc.fit_transform(temperatures_df[['TBh', 'TBv', 'TB_IR']])

#create the array I need to run KMeans
X2 = temperatures_df[['T_TBh', 'T_TBv', 'T_TB_IR']].values

#find the number of clusters.
optimise_k_means(temperatures_df[['T_TBh', 'T_TBv', 'TB_IR']], 20)

Since the number of clusters is a bit ambiguous, I run a ton of tests in the below cell to determine the best number.

In [ ]:
#10
km = KMeans(n_clusters=10, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0,
            algorithm='elkan')

y_km = km.fit_predict(X2)
# print("score for 10 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#9
km = KMeans(n_clusters=9, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 9 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#8
km = KMeans(n_clusters=8, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0,
            algorithm='elkan')

y_km = km.fit_predict(X2)
# print("score for 8 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#7
km = KMeans(n_clusters=7, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 7 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#6
km = KMeans(n_clusters=6, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 6 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#5
km = KMeans(n_clusters=5, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 5 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#4
km = KMeans(n_clusters=4, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 4 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#3
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X2)
# print("score for 3 clusters: ", silhouette_score(X2, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

None of these did particularly well. The best was using 3 clusters again.
- score for 3 clusters:  0.5608960051242237
    - 1  ||  86652
    - 2  ||  63177
    - 0 ||   32590

In [ ]:

my_map = folium.Map(location = center_of_map,
                   zoom_start = 7,
                   width = '90%',
                   height = '100%',
                   left = '5%',
                   right = '5%',
                   top = '0%',
                    tiles="CartoDB Positron") 
for index, row in map_df.iterrows():
    if row['cluster'] == 0:
        color = 'green'
    elif row['cluster'] == 2:
        color = 'brown'       
    else:
        color = 'blue' 
    folium.Circle(location=[row['footprint_lat'], row['footprint_lon']],
                  radius=50,
                  color=color,
                  fill=True,
                  fill_color=color,
                  fill_opacity=0.5).add_to(my_map)
my_map.save(path + 'Dots_map_TBIR.html')
my_map

In [ ]:
#create silhoutte map on the best of the TB_IR, TBv, and TBh dataset
# cluster_labels = np.unique(y_km)
# n_clusters = cluster_labels.shape[0]
# silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
# y_ax_lower, y_ax_upper = 0, 0
# yticks = []
# for i, c in enumerate(cluster_labels):
#     c_silhouette_vals = silhouette_vals[y_km == c]
#     c_silhouette_vals.sort()
#     y_ax_upper += len(c_silhouette_vals)
#     color = cm.jet(float(i) / n_clusters)
#     plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
#              edgecolor='none', color=color)

#     yticks.append((y_ax_lower + y_ax_upper) / 2.)
#     y_ax_lower += len(c_silhouette_vals)
    
# silhouette_avg = np.mean(silhouette_vals)
# plt.axvline(silhouette_avg, color="red", linestyle="--") 

# plt.yticks(yticks, cluster_labels + 1)
# plt.ylabel('Cluster')
# plt.xlabel('Silhouette coefficient')

# plt.tight_layout()
# plt.savefig('silhoutte plot 1.png', dpi=300)
# plt.show()

## TB_IR only

I wanted to see if I just used the infrared temperature if it might give better results.

In [ ]:
#create the array I need to run KMeans. No need for the transformed data since I am only using the one column.
X2 = temperatures_df[['TB_IR']].values

#find the number of clusters.
optimise_k_means(temperatures_df[[ 'TB_IR']], 20)

In [ ]:
#3
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X3)
# print("score for 3 clusters: ", silhouette_score(X3, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())


#4
km = KMeans(n_clusters=4, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X3)
# print("score for 4 clusters: ", silhouette_score(X3, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#5
km = KMeans(n_clusters=5, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X3)

# print("score for 5 clusters: ", silhouette_score(X3, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

In [ ]:
#since non of the above did very well, may as well try a couple higher numbers
#6
km = KMeans(n_clusters=6, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X3)
# print("score for 6 clusters: ", silhouette_score(X3, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())


#7
km = KMeans(n_clusters=7, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X3)
# print("score for 7 clusters: ", silhouette_score(X3, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

The best model for only using TB_IR was using 4 clusters. I am not going to bother creating a map for it.
- score for 4 clusters:  0.5517634627450231
    - 0    62138
    - 3    54114
    - 1    39146
    - 2    27021

## All df_high_Kscore
Testing KMeans on the entire dataset of high scoring variables. This was found using SelectKBest at the beginning of this project. 

In [ ]:
#transform all the variables in df_high_Kscore. Keeping the names the same.
df_high_Kscore[['TBh', 'TBv', 'footprint_lon', 'footprint_lat', 'footprint_semiminor', 'footprint_semimajor', 'aircraft_alt_AGL',   
               'aircraft_alt_MSL']] = sc.fit_transform(df_high_Kscore[['TBh', 'TBv', 'footprint_lon','footprint_lat', 
                                                                       'footprint_semiminor','footprint_semimajor',
                                                                       'aircraft_alt_AGL','aircraft_alt_MSL']])
#create the array needed.
X4 = df_high_Kscore.values

#find the best number of clusters
optimise_k_means(X4, 20)

Almost clearly 3 clusters, but I want to double-check using a couple higher numbers.

In [ ]:
#3
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X4)
# print("score for 3 clusters: ", silhouette_score(X4, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())


#4
km = KMeans(n_clusters=4, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X4)

# print("score for 4 clusters: ", silhouette_score(X4, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

#5
km = KMeans(n_clusters=5, 
            init='k-means++', 
            n_init=3, 
            max_iter=500,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X4)

# print("score for 5 clusters: ", silhouette_score(X4, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

Using 3 clusters wins again! 
- score for 3 clusters:  0.6839768835574461
    - 0    90427
    - 2    60989
    - 1    31003

# Final/Best Dataset

In [ ]:
X = df_high_Kscore[['T_TBh', 'T_TBv']].values

## Best model
This model had a silhoutte score of 0.828. Although the thickness varies greatly, I believe this to be expected due to the number of readings on dry land versus wet land or water. 

In [ ]:
km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=3, 
            max_iter=100,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(X)
# print("max iteration '100' change score: ", silhouette_score(X, y_km))
lables = pd.DataFrame(km.labels_)
print(lables.value_counts())

In [ ]:
center_of_map = 40.296669047966525, 0.346046919653026

my_map = folium.Map(location = center_of_map,
                   zoom_start = 14,
                   width = '90%',
                   height = '100%',
                   left = '5%',
                   right = '5%',
                   top = '0%',
                    tiles="CartoDB Positron") 
for index, row in map_df.iterrows():
    if row['cluster'] == 2:
        color = 'green'
    elif row['cluster'] == 1:
        color = 'brown'       
    else:
        color = 'blue' 
    folium.Circle(location=[row['footprint_lat'], row['footprint_lon']],
                  radius=50,
                  color=color,
                  fill=True,
                  fill_color=color,
                  fill_opacity=0.5).add_to(my_map)
my_map.save(path + 'Dots_map_final.html')
my_map

In [ ]:
# cluster_labels = np.unique(y_km)
# n_clusters = cluster_labels.shape[0]
# silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
# y_ax_lower, y_ax_upper = 0, 0
# yticks = []
# for i, c in enumerate(cluster_labels):
#     c_silhouette_vals = silhouette_vals[y_km == c]
#     c_silhouette_vals.sort()
#     y_ax_upper += len(c_silhouette_vals)
#     color = cm.jet(float(i) / n_clusters)
#     plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
#              edgecolor='none', color=color)

#     yticks.append((y_ax_lower + y_ax_upper) / 2.)
#     y_ax_lower += len(c_silhouette_vals)
    
# silhouette_avg = np.mean(silhouette_vals)
# plt.axvline(silhouette_avg, color="red", linestyle="--") 

# plt.yticks(yticks, cluster_labels + 1)
# plt.ylabel('Cluster')
# plt.xlabel('Silhouette coefficient')

# plt.tight_layout()
# plt.savefig('silhoutte plot 1.png', dpi=300)
# plt.show()

# Discussion

I am fairly happy with how KMeans split the clusters. Because it is unsupervised learning, there is no way to determine just how well it' performing. With this dataset, you can map it, but even that does not explain 100%. I do think that we can use it to train a supervised learning model with a bit of work. For instance, you could take a range of lat/lon that you can work with and fine-tune it. 

It was impressive to see that it was able to pick up on what was clearly water and land. The shoreline, though, might be a little ambiguous since it was picking up areas of warmer water as well as farmlands. For future projects, if the SLAP campaigns travel over forested areas, it would be interesting to see how KMeans might handle it since those areas tend to be a bit cooler than flat, open fields.

Other methods to try: DBSCAN, and EM learning.
- I did try to use DBSCAN on the dataset, but ran into memory issues, and I felt splitting the data was not the best method.
- For EM, I found no easy way to use it in Python, but I feel it might be able to provide a better model since it uses statitical methods versus euclidian distance.